In [1]:
import numpy as np
import pandas as pd
import os
import cv2
import torch
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import ImageFolder
from torchvision.models import resnet50
from tqdm import tqdm

In [2]:
# Custom Dataset class which applies transformation 
class CustomDataset(Dataset):
    def __init__(self, metadata, image_dir, transform=None):
        self.metadata = metadata
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_dir, self.metadata.iloc[idx, 0])
        image = cv2.imread(img_name)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 

        if self.transform:
            image = self.transform(image)

        label = self.metadata.iloc[idx, 1]
        return image, label

In [3]:

train_image_dir = 'sampled_data/train_data'
val_image_dir = 'sampled_data/val_data'
test_image_dir = 'sampled_data/test_data'


train_metadata = pd.read_csv('sampled_data/train_data.csv', usecols=['filename', 'age'])
val_metadata = pd.read_csv('sampled_data/val_data.csv', usecols=['filename', 'age'])
test_metadata = pd.read_csv('sampled_data/test_data.csv', usecols=['filename', 'age'])


In [4]:
# Image transformations with augmentation for training set so it can perform well on unseen data
train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees=10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Image transformations without augmentation for validation and test sets
val_test_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [5]:

train_dataset = CustomDataset(train_metadata, train_image_dir, transform=train_transform)
val_dataset = CustomDataset(val_metadata, val_image_dir, transform=val_test_transform)
test_dataset = CustomDataset(test_metadata, test_image_dir, transform=val_test_transform)


train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

In [6]:
# Load pre-trained ResNet-50 model
model = models.resnet50(pretrained=True)

# Modify the last fully connected layer to output a single value (age prediction)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 1)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

/opt/homebrew/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [8]:
# Training the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), targets.float())  # Assuming targets are integers, convert to float
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}")


Epoch [1/10], Loss: 250.41588101310526
Epoch [2/10], Loss: 169.7809436002517
Epoch [3/10], Loss: 150.2827923412629
Epoch [4/10], Loss: 137.1509117820046
Epoch [5/10], Loss: 126.27296443266027
Epoch [6/10], Loss: 120.26691058561764
Epoch [7/10], Loss: 111.59561193537584
Epoch [8/10], Loss: 102.87378977607278
Epoch [9/10], Loss: 98.58286336399017
Epoch [10/10], Loss: 95.92880350204713


In [9]:
# Validation
model.eval()  
val_loss = 0.0
total_correct_val = 0
total_samples_val = 0
with torch.no_grad():
    for inputs, targets in val_loader:
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), targets.float())
        val_loss += loss.item()

        # Calculate validation accuracy
        predictions_val = torch.round(outputs).squeeze()
        correct_val = (predictions_val == targets).sum().item()
        total_correct_val += correct_val
        total_samples_val += targets.size(0)

val_loss /= len(val_loader)
print(f"Validation Loss: {val_loss}, Validation Accuracy: {total_correct_val / total_samples_val}")


# Evaluation on Test Set
test_loss = 0.0
total_correct_test = 0
total_samples_test = 0
predictions_test = []
with torch.no_grad():
    for inputs, targets in test_loader:
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), targets.float())
        test_loss += loss.item()

        # Calculate test accuracy
        predictions = torch.round(outputs).squeeze()
        correct = (predictions == targets).sum().item()
        total_correct_test += correct
        total_samples_test += targets.size(0)
        

test_loss /= len(test_loader)
print(f"Test Loss: {test_loss}, Test Accuracy: {total_correct_test / total_samples_test}")

Validation Loss: 111.94293610524323, Validation Accuracy: 0.04332801701222754
Test Loss: 105.4825418993958, Test Accuracy: 0.045466702326825356
